# MODELS BABYYYYY - Sentiment Analysis - Big Richard Club

In [1]:
import pandas as pd
import numpy as np
import re 
import nltk 
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
import collections

from io import StringIO
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics

from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.metrics import classification_report,confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.patches as mpatches

plt.style.use('ggplot')
%matplotlib inline
pd.options.mode.chained_assignment = None  # default='warn'

## I. Import Data

In [2]:
x_train_bow = np.loadtxt("data/lesser_x_train_bow.txt.gz")
y_train = np.loadtxt("data/y_train.txt.gz")

print(f"X_train shape : {x_train_bow.shape} \n Y_train shape : {y_train.shape}")

X_train shape : (41157, 1000) 
 Y_train shape : (41157,)


In [3]:
x_val_bow = np.loadtxt("data/lesser_x_val_bow.txt.gz")
y_val = np.loadtxt("data/y_val.txt.gz")

print(f"x_val shape : {x_val_bow.shape} \n y_val shape : {y_val.shape}")

x_val shape : (3798, 1000) 
 y_val shape : (3798,)


# II. Model Charlotte

In [4]:
callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=5,
    restore_best_weights=True
)

In [5]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(
        input_dim=1000, # vocab of 1000 words
        output_dim=3 # 3 classes
    ), 
    tf.keras.layers.Dropout(0.5),
    
    tf.keras.layers.Conv1D(128, 5, activation = 'relu'),
    tf.keras.layers.MaxPooling1D(pool_size = 1),
    tf.keras.layers.Dropout(0.3),
    
    #tf.keras.layers.Conv1D(256, 5, activation = 'relu'),
    #tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dense(3, activation = 'softmax')
])
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 3)           3000      
                                                                 
 dropout (Dropout)           (None, None, 3)           0         
                                                                 
 conv1d (Conv1D)             (None, None, 128)         2048      
                                                                 
 max_pooling1d (MaxPooling1D  (None, None, 128)        0         
 )                                                               
                                                                 
 dropout_1 (Dropout)         (None, None, 128)         0         
                                                                 
 bidirectional (Bidirectiona  (None, 64)               31104     
 l)                                                     

In [6]:
history = model.fit(x_train_bow, y_train)

  46/1287 [>.............................] - ETA: 11:13 - loss: 1.0662 - accuracy: 0.4192

KeyboardInterrupt: 

# 44% ACCURACY C'EST CLAQUEEEEE

# SVM

In [7]:
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn import svm

In [8]:
linear_svm = svm.LinearSVC(max_iter=10000).fit(x_train_bow, y_train)

In [9]:
svm_training_accuracy = round(
    linear_svm.score(x_train_bow, y_train),
    5
)

print(f"Linear SVM Training Accuracy : {svm_training_accuracy * 100} %")

Linear SVM Training Accuracy : 75.518 %


In [10]:
svm_validation_accuracy = round(
    linear_svm.score(x_val_bow, y_val),
    5
)

print(f"Linear SVM validation Accuracy : {svm_validation_accuracy * 100} %")

Linear SVM validation Accuracy : 36.598000000000006 %


# SGD 

In [ ]:
from sklearn.linear_model import SGDClassifier

In [ ]:
sgd = SGDClassifier(
    early_stopping=False,
    penalty="l2",
    loss="hinge",
    max_iter=10000
).fit(x_train_bow, y_train)

In [ ]:
sgd_training_accuracy = round(
    sgd.score(x_train_bow, y_train),
    5
)

print(f"SGD Training Accuracy : {sgd_training_accuracy * 100} %")

In [ ]:
sgd_validation_accuracy = round(
    sgd.score(x_val_bow, y_val),
    5
)

print(f"SGD validation Accuracy : {sgd_validation_accuracy * 100} %")

# SVC

In [11]:
x_train_bow.shape

(41157, 1000)

In [16]:
nu_svc = svm.NuSVC(
    max_iter=1000
).fit(x_train_bow[:10000], y_train[:10000])

E:\miniconda3\envs\math4ml\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'


In [17]:
nu_svc_training_accuracy = round(
    nu_svc.score(x_train_bow[:10000], y_train[:10000]),
    5
)

print(f"Nu SVC Training Accuracy : {nu_svc_training_accuracy * 100} %")

Nu SVC Training Accuracy : 89.99000000000001 %


In [18]:
nu_svc_validation_accuracy = round(
    nu_svc.score(x_val_bow[:10000], y_val[:10000]),
    5
)

print(f"Nu SVC validation Accuracy : {nu_svc_validation_accuracy * 100} %")

Nu SVC validation Accuracy : 36.94 %


## Logistic Regression

In [37]:
# solver{‘newton-cg’, ‘lbfgs’, ‘liblinear’, ‘sag’, ‘saga’}

log_reg = LogisticRegression(
    penalty="l2",
    solver="lbfgs"
).fit(x_train_bow, y_train)

E:\miniconda3\envs\math4ml\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [38]:
log_reg_training_accuracy = round(
    log_reg.score(x_train_bow, y_train),
    5
)

print(f"Logistic Regression Training Accuracy : {log_reg_training_accuracy * 100} %")

Logistic Regression Training Accuracy : 75.58399999999999 %


In [39]:
log_reg_validation_accuracy = round(
    log_reg.score(x_val_bow, y_val),
    5
)

print(f"Logistic Regression validation Accuracy : {log_reg_validation_accuracy * 100} %")

Logistic Regression validation Accuracy : 36.414 %
